# ✨ WhatsApp Auto Blast ✨
---

In [97]:
# === LIBRARY ====

import os
import csv
import time
import string
import pyautogui
import random
import pywhatkit.whats as pw
import tkinter as tk
from tkinter import messagebox
from utils.wa_utils import detect_whatsapp_error, fixNumber, log_status

In [ ]:
# === MESSAGE TEMPLATE ===

HELLO: str = """\
Halo {name}, Kenalin aku Ryan IF-158 👋
"""

CUSTOM: str = """\
Custom
"""

In [ ]:
# === CONFIGURATION === 
# (Just Modify Here)

message_template: str = HELLO

delay_awal: int = random.randint(10, 12)
delay_akhir: int = random.randint(2, 3)

In [ ]:
# === ACTION ===

# Path Configuration
phone_column: str = "number"
file_path: str = "input.csv"

# Read CSV file
with open(file_path, "r", encoding="utf-8") as file:
    reader = csv.DictReader(file)
    for row in reader:

        # Check if phone column exists and is not empty
        if phone_column not in row:
            status: str = f"⚠️ ERROR: Column \'{phone_column}\' not found in CSV header."
            log_status("...", status)
            continue
        elif not row[phone_column].strip():
            status: str = f"⚠️ ERROR: Column \'{phone_column}\' value is empty."
            log_status("...", status)
            continue
        
        # Fix number format
        number: str = fixNumber(row[phone_column])

        # Check for required keys in the message template
        required_keys: list[str] = [field_name[1] for field_name in string.Formatter().parse(message_template) if field_name[1]]

        missing_keys: set[str] = set()
        empty_keys: set[str] = set()

        for key in required_keys:
            if key not in row:
                missing_keys.add(key)
            elif not row[key].strip():
                empty_keys.add(key)

        if missing_keys:
            status: str = f"⚠️ ERROR: Missing column(s) in CSV header: {', '.join(sorted(missing_keys))}"
            log_status(number, status)
            continue

        if empty_keys:
            status: str = f"⚠️ ERROR: Empty value(s) in row for column(s): {', '.join(sorted(empty_keys))}"
            log_status(number, status)
            continue
        
        # Make the message
        msg: str = message_template.format(**row)

        # Send message
        pw.sendwhatmsg_instantly(number, msg, wait_time=delay_awal, tab_close=False, close_time=delay_akhir)
        time.sleep(random.uniform(2.5, 4.5))

        # Check for possible errors
        status: str = detect_whatsapp_error(file_path)
        
        # Close tab
        time.sleep(random.uniform(0.4, 0.8))
        pyautogui.hotkey('ctrl', 'w')

        # Logging
        log_status(number, status)
        
# Cleanup
if os.path.exists("PyWhatKit_DB.txt"):
    os.remove("PyWhatKit_DB.txt")

# Notification
root = tk.Tk()
root.withdraw()
messagebox.showinfo("Selesai", "Proses selesai!")